# Lecture 2: Code specialization

In lecture 1, we learned how the semantics of types affect performance — it was not *possible* to implement a C-speed `sum` function, for example, unless (a) the element type is attached to the array as a whole so that (b) the element data can be stored consecutively in memory and (b) the computional code can be specialized at compile-time for `+` on this element type.

For code to be fast, however, it must also be possible to take advantage of this information in your own code.  This is obviously possible in statically typed languages like C, which are compiled to machine instructions ahead of time with type information declared explicitly to the compiler.  It is more subtle in a language like Julia, where the programmer does *not* usually explicitly declare type information.

In particular, we will talk about how Julia achieves both fast code and *type-generic code* (e.g. a `sum` function that works on any container of any type supporting `+`, even user-defined types), by aggressive automated **code specialization** by the Julia compiler.

## A type-generic sum function

Let's again look at our simple hand-written `sum` function.

In [1]:
function mysum(a)
    s = zero(eltype(a))
    @simd for x in a
        s += x
    end
    return s
end

mysum (generic function with 1 method)

In [2]:
a = rand(10^7)
mysum(a)

5.000429501170924e6

In [3]:
mysum(a) ≈ sum(a)

true

In [4]:
using BenchmarkTools, Compat
@btime sum($a)
@btime mysum($a)

  3.769 ms (0 allocations: 0 bytes)
  4.367 ms (0 allocations: 0 bytes)


5.000429501170924e6

Hooray!  Basically the same speed as Julia's built-in `sum` function and `numpy.sum`!   And it only required **7 lines of code**, some care with types, and a very minor bit of wizardry with the `@simd` tag to get the last factor of two.

Moreover, the code is still **type generic**: it can sum any container of any type that works with addition.   For example, it works for complex numbers, which are about two times slower as you might expect (since each complex addition requires two real additions):

In [5]:
z = rand(Complex{Float64}, length(a));
@btime mysum($z)

  11.804 ms (0 allocations: 0 bytes)


4.997398024812783e6 + 4.999404270717948e6im

And we **didn't have to declare any types** of any arguments or variables; the compiler figured everything out.  How?

## Type inference and specialization

To go any further, you need to understand something very basic about how Julia works.  Suppose we define a very simple function:

In [6]:
f(x) = x + 1

f (generic function with 1 method)

We didn't declare the type of `x`, and so our function `f(x)` will work with **any type of `x`** (as long as the `+ 1` operation is defined for that type):

In [7]:
f(3) # x is an integer (technically, a 64-bit integer)

4

In [8]:
g(x) = x + 2

g (generic function with 1 method)

In [9]:
@btime g(3)

  1.893 ns (0 allocations: 0 bytes)


5

In [10]:
f(3.1) # x is a floating-point value (Float64)

4.1

In [11]:
f([1,2,3]) # x is an array of integers

3-element Array{Int64,1}:
 2
 3
 4

In [12]:
f("hello")

LoadError: [91mMethodError: no method matching +(::String, ::Int64)[0m
Closest candidates are:
  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424
  +([91m::Complex{Bool}[39m, ::Real) at complex.jl:247
  +([91m::Char[39m, ::Integer) at char.jl:40
  ...[39m

How can a function like `f(x)` work for any type?  In Python, `x` would be a "box" that could contain anything, and it would then look up *at runtime* how to compute `x + 1`.  But we just saw that untyped Julia `sum` code could be fast.

The secret is **just-in-time (JIT) compilation**.   The first time you call `f(x)` **with a new type of argument** `x`, it will **compile a new version of `f` specialized for that type**.  The *next* time it calls `f(x)` with the same argument type

So, right now, after evaluating the above code, we have *three* versions of `f` compiled and sitting in memory: one for `x` of type `Int` (we say `x::Int` in Julia), one for `x::Float64`, and one for `x::Vector{Int}`.

We can even see what the compiled code for `f(x::Int)` looks like, either the [compiler (LLVM) bytecode](https://en.wikipedia.org/wiki/LLVM) or the low-level (below C!) [assembly code](https://en.wikipedia.org/wiki/Assembly_language):

In [13]:
@code_llvm f(1)


define i64 @julia_f_61261(i64) #0 !dbg !5 {
top:
  %1 = add i64 %0, 1
  ret i64 %1
}


In [14]:
@code_native f(1)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[6]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	leaq	1(%rdi), %rax
	popq	%rbp
	retq
	nopw	(%rax,%rax)


Let's break this down.  When you tell Julia's compiler that `x` is an `Int`, it:

* It knows `x` fits into a 64-bit CPU register (and is passed to the function via a register).

* Looks at `x + 1`.  Since `x` and `1` are both `Int`, it knows it should call the `+` function for two `Int` values.  This corresponds to *one machine instruction* `leaq` to add two 64-bit registers.

* Since the `+` function here is so simple, it won't bother to do a function call.  It will [inline](https://en.wikipedia.org/wiki/Inline_expansion) the `(+)(Int,Int)` function into the compiled `f(x)` code.

* Since it now knows what `+` function it is calling, it knows that the *result* of the `+` is *also* an `Int`, and it can return it via register.

This process works recursively if we define a new function `g(x)` that calls `f(x)`:

In [15]:
g(x) = f(x) + 3
g(1)

5

In [16]:
@code_llvm g(1)


define i64 @julia_g_61462(i64) #0 !dbg !5 {
top:
  %1 = add i64 %0, 4
  ret i64 %1
}


When it specialized `g` for `x::Int`, it not only figured out what `f` function to call, it not only *inlined* `f(x)` *into `g`*, but the compiler was smart enough to *combine the two additions* into a single addition `x + 4`.

## Defeating type inference: Type instabilities

To get good performance, there are some fairly simple rules that you need to follow in Julia code to avoid defeating the compiler's type inference.   See also the [performance tips section of the Julia manual](http://docs.julialang.org/en/stable/manual/performance-tips/).

Three of the most important are:

* Don't use (non-constant) global variables in critical code — put your critical code into a function (this is good advice anyway, from a software-engineering standpoint).  The compiler assumes that a **global variable can change type at any time**, so it is always stored in a "box", and "taints" anything that depends on it.

* Local variables should be "type-stable": **don't change the type of a variable inside a function**.  Use a new variable instead.

* Functions should be "type-stable": **a function's return type should only depend on the argument types, not on the argument values**.

To diagnose all of these problems, the `@code_warntype` macro that we used above is your friend.  If it labels any variables (or the function's return value) as `Any` or `Union{...}`, it means that the compiler couldn't figure out a precise type.

The third point, type-stability of functions, leads to lots of important but subtle choices in library design.  For example, consider the (built-in) `sqrt(x)` function, which computes $\sqrt{x}$:

In [17]:
sqrt(4)

2.0

You might think that `sqrt(-1)` should return $i$ (or `im`, in Julia syntax).  (Matlab's `sqrt` function does this.)  Instead, we get:

In [18]:
sqrt(-1)

LoadError: DomainError:
sqrt will only return a complex result if called with a complex argument. Try sqrt(complex(x)).

In [19]:
sqrt(-1 + 0im)

0.0 + 1.0im

Why did Julia implement `sqrt` in this silly way, throwing an error for negative arguments unless you add a zero imaginary part?  Any reasonable person wants an imaginary result from `sqrt(-1)`, surely?

The problem is that defining `sqrt` to return an imaginary result from `sqrt(-1)` would **not be type stable**: `sqrt(x)` would return a real result for non-negative real `x`, and a complex result for negative real `x`, so the **return type would depend on the value of `x`** and **not just its type.**

That would defeat type inference, not just for the `sqrt` function, but for **anything the sqrt function touches**.  Unless the compiler can somehow figure out `x ≥ 0`, it will have to either store the result in a "box" or compile two branches of the result.  Let's see how that works by defining our own square-root function:

In [20]:
mysqrt(x::Complex) = sqrt(x)
mysqrt(x::Real) = x < 0 ? sqrt(complex(x)) : sqrt(x)

mysqrt (generic function with 2 methods)

This definition is an example of Julia's [multiple dispatch style](http://docs.julialang.org/en/stable/manual/methods/), which in some sense is a generalization of object-oriented programming but focuses on "verbs" (functions) rather than nouns.  We will discuss this more in a later lecture.

The `::Complex` and `::Real` are argument-type declarations.  Such declarations are **not related to performance**, but instead **act as a "filter"** to allow us to have one version of `mysqrt` for complex arguments and another for real arguments.

In [21]:
mysqrt(2)

1.4142135623730951

In [22]:
mysqrt(-2)

0.0 + 1.4142135623730951im

In [23]:
mysqrt(-2+0im)

0.0 + 1.4142135623730951im

Looks great, right?  But let's see what happens to type inference in a function that calls `mysqrt` instead of `sqrt`:

In [24]:
slowfun(x) = mysqrt(x) + 1
@code_warntype slowfun(2)

Variables:
  #self#::#slowfun
  x::Int64
  #temp#@_3::Union{Complex{Float64}, Float64}
  #temp#@_4::Core.MethodInstance
  #temp#@_5::Union{Complex{Float64}, Float64}

Body:
  begin 
      $(Expr(:inbounds, false))
      # meta: location In[20] mysqrt 2
      unless (Base.slt_int)(x::Int64, 0)::Bool goto 6
      #temp#@_3::Union{Complex{Float64}, Float64} = $(Expr(:invoke, MethodInstance for sqrt(::Complex{Float64}), :(Base.sqrt), :($(Expr(:new, Complex{Float64}, :((Base.sitofp)(Float64, x)::Float64), :((Base.sitofp)(Float64, 0)::Float64))))))
      goto 8
      6: 
      #temp#@_3::Union{Complex{Float64}, Float64} = (Base.Math.sqrt_llvm)((Base.sitofp)(Float64, x::Int64)::Float64)::Float64
      8: 
      # meta: pop location
      $(Expr(:inbounds, :pop))
      unless (#temp#@_3::Union{Complex{Float64}, Float64} isa Complex{Float64})::Bool goto 14
      #temp#@_4::Core.MethodInstance = MethodInstance for +(::Complex{Float64}, ::Int64)
      goto 23
      14: 
      unless (#temp#@_3::U

Because the compiler **doesn't know at compile-time that x is positive** (at compile-time it **uses only types, not values**, it doesn't know whether the result is real (`Float64`) or complex (`Complex{Float64}`) and has to store it in a "box".  This kills performance.

## Defining our own types

Let's define our own type to represent a **"point" in two dimensions**.  Each point will have an $(x,y)$ location.  So that we can use the points with our `sum` functions above, we'll also define `+` and `zero` functions to do the obvious **vector addition**.

One such definition in Julia is:

In [25]:
mutable struct Point1
    x
    y
end
Base.:+(p::Point1, q::Point1) = Point1(p.x + q.x, p.y + q.y)
Base.zero(::Type{Point1}) = Point1(0,0)

Point1(3,4)

Point1(3, 4)

In [26]:
Point1(3,4) + Point1(5,6)

Point1(8, 10)

Our type is very generic, and can hold any type of `x` and `y` values:

In [27]:
Point1(3.7, 4+5im)

Point1(3.7, 4 + 5im)

Perhaps too generic:

In [28]:
Point1("x", [3,4,5])

Point1("x", [3, 4, 5])

Since `x` and `y` can be *anything*, they must be **pointers to "boxes"**.  This is **bad news for performance**.

A `mutable struct` is *mutable*, which means we can create a `Point1` object and then change `x` or `y`:

In [29]:
p = Point1(3,4)
p.x = 7
p

Point1(7, 4)

This means that every reference to a `Point1` object must be a *pointer* to an object stored elsewhere in memory, because *how else would we "know" when an object changes?*  Furthermore, an **array of `Point1` objects must be an array of pointers** (which is **bad news for performance** again):

In [30]:
P = [p,p,p]

3-element Array{Point1,1}:
 Point1(7, 4)
 Point1(7, 4)
 Point1(7, 4)

In [31]:
p.y = 8
P

3-element Array{Point1,1}:
 Point1(7, 8)
 Point1(7, 8)
 Point1(7, 8)

Let's test this out by creating an array of `Point1` objects and summing it.  Ideally, this would be about twice as slow as summing an equal-length array of numbers, since there are twice as many numbers to sum.  But because of all of the boxes and pointer-chasing, it should be far slower.

To create the array, we'll call the `Point1(x,y)` constructor with our array `a`, using Julia's ["dot-call" syntax](http://docs.julialang.org/en/stable/manual/functions/#dot-syntax-for-vectorizing-functions) that applies a function "element-wise" to arrays:

In [32]:
a1 = Point1.(a, a)

10000000-element Array{Point1,1}:
 Point1(0.963508, 0.963508)    
 Point1(0.28758, 0.28758)      
 Point1(0.909506, 0.909506)    
 Point1(0.523264, 0.523264)    
 Point1(0.320009, 0.320009)    
 Point1(0.0172728, 0.0172728)  
 Point1(0.658692, 0.658692)    
 Point1(0.175798, 0.175798)    
 Point1(0.150886, 0.150886)    
 Point1(0.00902187, 0.00902187)
 Point1(0.167971, 0.167971)    
 Point1(0.587693, 0.587693)    
 Point1(0.40574, 0.40574)      
 ⋮                             
 Point1(0.560274, 0.560274)    
 Point1(0.512661, 0.512661)    
 Point1(0.603508, 0.603508)    
 Point1(0.786933, 0.786933)    
 Point1(0.651256, 0.651256)    
 Point1(0.590537, 0.590537)    
 Point1(0.397866, 0.397866)    
 Point1(0.763691, 0.763691)    
 Point1(0.536594, 0.536594)    
 Point1(0.184702, 0.184702)    
 Point1(0.0561824, 0.0561824)  
 Point1(0.750536, 0.750536)    

In [33]:
@btime sum($a1)

  581.316 ms (29999997 allocations: 610.35 MiB)


Point1(5.000429501170885e6, 5.000429501170885e6)

In [34]:
@btime mysum($a1)

  607.055 ms (30000001 allocations: 610.35 MiB)


Point1(5.000429501170522e6, 5.000429501170522e6)

The time is at least **50× slower** than we would like, but consistent with our other timing results on "boxed" values from last lecture.

### An imperfect solution: A concrete immutable type

We can avoid these two problems by:

* Declare the types of `x` and `y` to be *concrete* types, so that they don't need to be pointers to boxes.
* Declare our Point to be an [immutable](https://en.wikipedia.org/wiki/Immutable_object) type (`x` and `y` cannot change), so that Julia is not forced to make every reference to a Point into a pointer: just `struct`, not `mutable struct`:

In [36]:
struct Point2
    x::Float64
    y::Float64
end
Base.:+(p::Point2, q::Point2) = Point2(p.x + q.x, p.y + q.y)
Base.zero(::Type{Point2}) = Point2(0.0,0.0)

Point2(3,4)

Point2(3.0, 4.0)

In [37]:
Point2(3,4) + Point2(5,6)

Point2(8.0, 10.0)

In [38]:
p = Point2(3,4)
P = [p,p,p]

3-element Array{Point2,1}:
 Point2(3.0, 4.0)
 Point2(3.0, 4.0)
 Point2(3.0, 4.0)

In [39]:
p.x = 6 # gives an error since p is immutable

LoadError: [91mtype Point2 is immutable[39m

If this is working as we hope, then summation should be much faster:

In [40]:
a2 = Point2.(a,a)
@btime sum($a2)

  11.950 ms (0 allocations: 0 bytes)


Point2(5.000429501170885e6, 5.000429501170885e6)

Now the time is **only about 10ms**, only slightly more than twice the cost of summing an array of individual numbers of the same length!

Unfortunately, we paid a big price for this performance: our `Point2` type only works with *a single numeric type* (`Float64`), much like a C implementation.

### The best of both worlds: Parameterized immutable types

How do we get a `Point` type that works for *any* type of `x` and `y`, but at the same time allows us to have an array of points that is concrete and homogeneous (every point in the array is forced to be the same type)?  At first glance, this seems like a contradiction in terms.

The answer is not to define a *single* type, but rather to **define a whole family of types** that are *parameterized* by the type `T` of `x` and `y`.  In computer science, this is known as [parametric polymorphism](https://en.wikipedia.org/wiki/Parametric_polymorphism).  (An example of this can be found in [C++ templates](https://en.wikipedia.org/wiki/Template_%28C%2B%2B%29).)

In Julia, we will define such a family of types as follows:

In [41]:
struct Point3{T<:Real}
    x::T
    y::T
end
Base.:+(p::Point3, q::Point3) = Point3(p.x + q.x, p.y + q.y)
Base.zero{T}(::Type{Point3{T}}) = Point3(zero(T),zero(T))

Point3(3,4)

Point3{Int64}(3, 4)

Here, `Point3` is actually a family of subtypes `Point{T}` for different types `T`.   The notation `<:` in Julia means "is a subtype of", and hence `T<:Real` means that we are constraining `T` to be a `Real` type (a built-in *abstract type* in Julia that includes e.g. integers or floating point).

In [42]:
Point3(3,4) + Point3(5.6, 7.8)

Point3{Float64}(8.6, 11.8)

Now, let's make an array:

In [43]:
a3 = Point3.(a,a)

10000000-element Array{Point3{Float64},1}:
 Point3{Float64}(0.963508, 0.963508)    
 Point3{Float64}(0.28758, 0.28758)      
 Point3{Float64}(0.909506, 0.909506)    
 Point3{Float64}(0.523264, 0.523264)    
 Point3{Float64}(0.320009, 0.320009)    
 Point3{Float64}(0.0172728, 0.0172728)  
 Point3{Float64}(0.658692, 0.658692)    
 Point3{Float64}(0.175798, 0.175798)    
 Point3{Float64}(0.150886, 0.150886)    
 Point3{Float64}(0.00902187, 0.00902187)
 Point3{Float64}(0.167971, 0.167971)    
 Point3{Float64}(0.587693, 0.587693)    
 Point3{Float64}(0.40574, 0.40574)      
 ⋮                                      
 Point3{Float64}(0.560274, 0.560274)    
 Point3{Float64}(0.512661, 0.512661)    
 Point3{Float64}(0.603508, 0.603508)    
 Point3{Float64}(0.786933, 0.786933)    
 Point3{Float64}(0.651256, 0.651256)    
 Point3{Float64}(0.590537, 0.590537)    
 Point3{Float64}(0.397866, 0.397866)    
 Point3{Float64}(0.763691, 0.763691)    
 Point3{Float64}(0.536594, 0.536594)    
 Point3{Float6

Note that the type of this array is `Array{Point3{Float64},1}` (we could equivalently write this as `Vector{Point3{Float64}}`, since `Vector{T}` is a synonym for `Array{T,1}`).  You should learn a few things from this:

* An `Array{T,N}` in Julia is itself a parameterized type, parameterized by the element type `T` and the dimensionality `N`.

* Since the element type `T` is encoded in the `Array{T,N}` type, the element type does not need to be stored in each element.  That means that the `Array` is free to store an array of "inlined" elements, rather than an array of pointers to boxes.  (This is why `Array{Float64,1}` earlier could be stored in memory like a C `double*`.

* It is still important that the element type be `immutable`, since an array of mutable elements would still need to be an array of pointers (so that it could "notice" if another reference to an element mutates it).

In [44]:
@btime sum($a3)
@btime mysum($a3)

  12.415 ms (0 allocations: 0 bytes)
  11.846 ms (0 allocations: 0 bytes)


Point3{Float64}(5.000429501170522e6, 5.000429501170522e6)

Hooray! It is again **only about 10ms**, the same time as our completely concrete and inflexible `Point2`.